<a href="https://colab.research.google.com/github/italomarcelogit/auto-sklearn_regressao/blob/main/auto_sklearn_exemplo_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics
from datetime import datetime
!pip install --upgrade -q pip
!pip install --upgrade -q scipy==1.7.1
!pip install -q auto-sklearn
import plotly.graph_objects as go

**IMPORTANTE:**
Após as importações e instalações anterior, reinicie o ambiente

In [ ]:
import autosklearn.regression

In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/italomarcelogit/scripts-diversos/master/dataset-sales-random/vendas-dataset-aleatorio.csv', sep=';')
df.head()

,ds,y,y1,y2,y3,y4,y5,y6
0,2000-01-01,750.0,100.0,100.0,150.0,150.0,130.0,120.0
1,2000-02-01,750.0,100.0,100.0,150.0,150.0,130.0,120.0
2,2000-03-01,1000.0,145.0,145.0,200.0,200.0,150.0,160.0
3,2000-04-01,1000.0,145.0,145.0,200.0,200.0,150.0,160.0
4,2000-05-01,1000.0,145.0,145.0,200.0,200.0,150.0,160.0


In [3]:
df2 = df[['ds', 'y']].copy()
df2['diaAno'] = df.ds.apply(lambda x: pd.to_datetime([x]).dayofyear[0])
df2['ano'] = df.ds.apply(lambda x: pd.to_datetime([x]).year[0])
df2.pop('ds')
df2.head()

,y,diaAno,ano
0,750.0,1,2000
1,750.0,32,2000
2,1000.0,61,2000
3,1000.0,92,2000
4,1000.0,122,2000


In [4]:
ano = 2021
Xtreino = df2[df2.ano<ano][['diaAno', 'ano']]
ytreino = df2[df2.ano<ano]['y']
Xteste = df2[df2.ano>=ano][['diaAno', 'ano']]
yteste = df2[df2.ano>=ano]['y']

In [5]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/content/autosklearn_regression_example_tmp',
)

In [ ]:
automl.fit(Xtreino, ytreino)

In [7]:
automl.leaderboard()

,rank,ensemble_weight,type,cost,duration
model_id,,,,,
2,1,0.34,random_forest,0.0,1.606499
42,2,0.40,gradient_boosting,0.0,1.028372
43,3,0.26,gaussian_process,0.0,1.289344


In [ ]:
automl.show_models()

In [9]:
df2.ano.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [10]:
label_treino = automl.predict(Xtreino)
label_teste = automl.predict(Xteste)

In [11]:
print(f"R2 score - treino: {sklearn.metrics.r2_score(ytreino, label_treino)}",
f"\nR2 score - teste: {sklearn.metrics.r2_score(yteste, label_teste)}")

R2 score - treino: 1.0 
R2 score - teste: 1.0


In [12]:
def scatter(data, x, y, txt_titulo='', txt_xaxis='', txt_yaxis=''):
  # fig = go.Figure(data=go.Scatter(x=df[x], y=df[y], mode='lines'))
  fig = go.Figure()
  for f in x:
    for l in y:
      fig.add_trace(go.Scatter(x=data[f], y=data[l],
                      mode='lines+markers',
                      name=l))
  fig.update_layout(title=txt_titulo,
                    xaxis_title=txt_xaxis,
                    yaxis_title=txt_yaxis,
                    template='plotly_white',
                    xaxis = dict(tickangle=80)
                    )
  fig.show()


In [13]:
vdc = []
for x in Xteste.index:
  a = Xteste.loc[x]['ano']
  da = Xteste.loc[x]['diaAno']
  dc = datetime.strptime(f'{a}' + "-" + f'{da}', "%Y-%j").strftime("%d-%m-%Y")
  vdc.append(dc)

previsao = pd.DataFrame()
previsao['Realizado'] = yteste.to_list()
previsao['Previsão'] = label_teste.tolist()
previsao['Período'] = vdc

In [14]:
scatter(previsao, x=['Período'], y=['Realizado', 'Previsão'],
        txt_titulo='AutoML')